    Roman Peerboom  10791523
    Floris de Vries 11710799

In [85]:
import numpy
import simpy
import random
import statistics 

In [236]:
lambda_1 = arrival_rate = 10              # Arrival rate, events in given time period
time_in_system = 20                       # Mean time in system 
mu = 1/time_in_system                     # How much people one server can help for a given time period 


In [486]:
def source(env, number, lambda_1, counter, mu):
    """Source generates customers randomly"""
    for i in range(number):
        c = object(env, 'Customer%02d' % i, counter, mu)
        env.process(c)
        t = 1-random.expovariate(-lambda_1)
        yield env.timeout(t)
        
        
def object(env, name, counter, mu):
    """Customer arrives, is served and leaves."""
    arrive = env.now
#     print('%7.4f %s: Here I am' % (arrive, name))
    with counter.request() as req:
        results = yield req
        wait = env.now - arrive
        waiting_times.append(wait)
        # We got to the counter
#         print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
        tib = 1- random.expovariate(-mu)
        yield env.timeout(tib)
#         print('%7.4f %s: Finished' % (env.now, name))


In [508]:
n = 1                                      # Number of servers
lambda_1 = arrival_rate = 1*n              # Arrival rate, events in given time period
time_in_system = 0.98                         # Mean time in system 
mu = 1/time_in_system                      # How much people one server can help for a given time period 

number_of_arrivals = 10000000000
print("\N{greek small letter rho} is: " + str(lambda_1/(n*mu)) + " for " + str(n) + " servers.")
simulations = 20
mean_waiting_times = []

for simulation in range(simulations):
    waiting_times = []
    env = simpy.Environment()
    server = simpy.Resource(env, capacity=n) 
    env.process(source(env,number_of_arrivals,lambda_1, server, mu))
    env.run(until=100000)
    print("Average waiting time for this simulation: " + str(statistics.mean(waiting_times)))
    mean_waiting_times.append(statistics.mean(waiting_times))
    
print("\nAverage waiting time for all simulations: " + str(statistics.mean(mean_waiting_times)) +
      " with sd: " + str(statistics.stdev(mean_waiting_times)) + " with \N{greek small letter rho} is: " 
      + str(lambda_1/(n*mu)) + " for " + str(n) + " servers.")

ρ is: 0.98 for 1 servers.
Average waiting time for this simulation: 47.14275085390751
Average waiting time for this simulation: 31.221310436751576
Average waiting time for this simulation: 33.19586305241329
Average waiting time for this simulation: 55.22007894127132
Average waiting time for this simulation: 29.786190352881018
Average waiting time for this simulation: 43.36534800551942
Average waiting time for this simulation: 43.94721112227724
Average waiting time for this simulation: 52.04876353944948
Average waiting time for this simulation: 38.500787166328
Average waiting time for this simulation: 64.23808466135122
Average waiting time for this simulation: 49.106567600196996
Average waiting time for this simulation: 77.32417650136192
Average waiting time for this simulation: 28.20172742720168
Average waiting time for this simulation: 59.58928128169268
Average waiting time for this simulation: 34.10582590418343
Average waiting time for this simulation: 71.49738737497435
Average waiti

In [495]:
n_s = [1, 2, 4]                              # Number of servers
simulations = 20
mean_waiting_times = [[],[],[]]

for n_ind in range(len(n_s)):
    n = n_s[n_ind]
    lambda_1 = arrival_rate = 1*n              # Arrival rate, events in given time period
    time_in_system = 0.98                         # Mean time in system 
    mu = 1/time_in_system                      # How much people one server can help for a given time period 
    number_of_arrivals = 10000000000
    print("\N{greek small letter rho} is: " + str(lambda_1/(n*mu)) + " for " + str(n) + " server(s).")
    for simulation in range(simulations):
#         print("Simulation: " + str(simulation))
        waiting_times = []
        env = simpy.Environment()
        server = simpy.Resource(env, capacity=n) 
        env.process(source(env,number_of_arrivals,lambda_1, server, mu))
        env.run(until=100000)
#         print("Average waiting time for this simulation: " + str(statistics.mean(waiting_times)))
        mean_waiting_times[n_ind].append(statistics.mean(waiting_times))    
    print("Average waiting time: " + str(statistics.mean(mean_waiting_times[n_ind])) +
          " with sd: " + str(statistics.stdev(mean_waiting_times[n_ind])) + "\n")

ρ is: 0.98 for 1 server(s).
Average waiting time: 46.97562033997208 with sd: 19.40511238168693

ρ is: 0.98 for 2 server(s).
Average waiting time: 0.12912325329840557 with sd: 0.0024490032562412833

ρ is: 0.98 for 4 server(s).
Average waiting time: 0.00013613556810785092 with sd: 3.611890560488397e-05



In [506]:
## Plotting the results as boxplots for different methods
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


df_1_server = pd.DataFrame({'Waiting time': mean_waiting_times[0], 'Method': ['1 server' for i in range(len(mean_waiting_times[0]))]})
df_2_server = pd.DataFrame({'Waiting time': mean_waiting_times[1], 'Method': ['2 servers' for i in range(len(mean_waiting_times[1]))]})
df_3_server = pd.DataFrame({'Waiting time': mean_waiting_times[2], 'Method': ['4 servers' for i in range(len(mean_waiting_times[2]))]})
df_total = df_1_server.append(df_2_server)
df_total = df_total.append(df_3_server)



sns.reset_orig ()

p = sns.catplot(x="Method", y="Waiting time", kind="box", data=df_total, height=6, aspect=12/6)

plt.xlabel("Amount of servers", size = 20)
plt.ylabel("Average waiting time", size = 20)
# plt.title("The area for different sampling methods", size=20, y= 1.05)
_, ylabels = plt.yticks()
_, xlabels = plt.xticks()
p.set_yticklabels(ylabels, size=15)
p.set_xticklabels(xlabels, size=15)
# p.set_yscale("log")
p.set(yscale="log")

plt.savefig('boxplots_server_amount.png', format='png', dpi=600, bbox_inches = "tight")


C:\Users\flori\anaconda3\lib\site-packages\seaborn\axisgrid.py:923: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(labels, **kwargs)
